Install gwnet package:

In [ ]:
!pip install --upgrade git+git://github.com/Sunnesoft/g2net-challenge.git

from gwnet import GwEfficientNetB0, GwXception, GwLeNet, TfDevice

Mount Google Drive folder with processed by whiten, bandpass and notch filters dataset:

In [ ]:
import os
from google.colab import drive

BASE_PATH = '/content/'
DRIVE_PATH = os.path.join(BASE_PATH, 'drive')
drive.mount(DRIVE_PATH)

Set up the configuration and extract the train dataset from Google Drive to $BASE_PATH folder:

In [ ]:
IMAGE_SIZE = (512, 512, 3)
IMAGE_SCALE = 255.0
BATCH_SIZE = 64
EPOCHS = 20

INPUT_PATH = os.path.join(BASE_PATH, 'tfrecords/train/')
ZIP_GD_IN_FILE = os.path.join(DRIVE_PATH, 'MyDrive/g2net/tfrecords_train.zip')
EXPER_TPL = f'{"_".join(IMAGE_SIZE)}_{BATCH_SIZE}_{EPOCHS}'
OUT_PATH = os.path.join(BASE_PATH, f'model/{EXPER_TPL}')
GD_OUT_PATH = os.path.join(DRIVE_PATH, f'MyDrive/g2net/model/{EXPER_TPL}/')

!mkdir -p $INPUT_PATH
!mkdir -p $GD_OUT_PATH
!unzip -qq $ZIP_GD_IN_FILE -d '/'

Fit EffitientNetB0 model:

In [1]:
solver = GwEfficientNetB0(
    name='eff_net_b0',
    mode=TfDevice.TPU,
    image_size=IMAGE_SIZE,
    image_scale_factor=IMAGE_SCALE,
    model_path=OUT_PATH)

solver.load_train_dataset(INPUT_PATH, batch_size=BATCH_SIZE)
solver.show_random_train_batch(subs={'1': 'GW_TRUE', '0': 'GW_FALSE'})

solver.compile()
solver.print_model()

solver.fit(epochs=EPOCHS)
solver.show_hist()

In [ ]:
command = f'cp {solver.model_fullpath} {os.path.join(GD_OUT_PATH, solver.model_filename)}'
!{command}

Fit Xception model:

In [ ]:
solver = GwXception(
    name='xception',
    mode=TfDevice.TPU,
    image_size=IMAGE_SIZE,
    image_scale_factor=IMAGE_SCALE,
    model_path=OUT_PATH)

solver.load_train_dataset(INPUT_PATH, batch_size=BATCH_SIZE)
solver.show_random_train_batch(subs={'1': 'GW_TRUE', '0': 'GW_FALSE'})

solver.compile()
solver.print_model()

solver.fit(epochs=EPOCHS)
solver.show_hist()

In [ ]:
command = f'cp {solver.model_fullpath} {os.path.join(GD_OUT_PATH, solver.model_filename)}'
!{command}

Fit LeNet model:

In [ ]:
solver = GwLeNet(
    name='le_net',
    mode=TfDevice.TPU,
    image_size=IMAGE_SIZE,
    image_scale_factor=IMAGE_SCALE,
    model_path=OUT_PATH)

solver.load_train_dataset(INPUT_PATH, batch_size=BATCH_SIZE)
solver.show_random_train_batch(subs={'1': 'GW_TRUE', '0': 'GW_FALSE'})

solver.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['AUC'])
solver.print_model()

solver.fit(epochs=EPOCHS)
solver.show_hist()

In [ ]:
command = f'cp {solver.model_fullpath} {os.path.join(GD_OUT_PATH, solver.model_filename)}'
!{command}